<a href="https://colab.research.google.com/github/mcnica89/MATH4060/blob/main/Week3B_time_to_hit_in_Gamblers_ruin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import itertools
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax import random as jrandom
from jax import nn as jnn
from jax import jit
import random
import time
import timeit
import math
import numpy as np

plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15)
font = {'size'   : 20}

plt.rc('font', **font)

jnp.set_printoptions(precision=3,suppress=True)

In [2]:
def gamblers_ruin_matrix(N_target):
  transition_matrix = np.zeros((N_target+1,N_target+1))
  i,j = np.indices(transition_matrix.shape)
  transition_matrix[i==j-1] = 0.5
  transition_matrix[i==j+1] = 0.5

  transition_matrix[0,0] = 1
  transition_matrix[0,1] = 0
  
  transition_matrix[N_target,N_target] = 1
  transition_matrix[N_target,N_target-1] = 0
  
  return transition_matrix

In [15]:
N_target = 10

In [16]:
P = gamblers_ruin_matrix(N_target)
print(P)

[[1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]]


In [23]:
M = np.eye(11)-P
print(M)

[[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [-0.5  1.  -0.5  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.  -0.5  1.  -0.5  0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  -0.5  1.  -0.5  0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  -0.5  1.  -0.5  0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.  -0.5  1.  -0.5  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.  -0.5  1.  -0.5  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5  0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5  0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]]


In [22]:
v = np.ones(N_target+1)
v[0] = 0
v[N_target] = 0
print(v)

[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]


In [24]:
jnp.linalg.solve(M,v)

DeviceArray([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [25]:
M_modified = M
M_modified[0,0]=1
M_modified[N_target,N_target]=1
print(M_modified)

[[ 1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [-0.5  1.  -0.5  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.  -0.5  1.  -0.5  0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  -0.5  1.  -0.5  0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  -0.5  1.  -0.5  0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.  -0.5  1.  -0.5  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.  -0.5  1.  -0.5  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5  0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5  0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1. ]]


In [26]:
jnp.linalg.solve(M_modified,v)

DeviceArray([ 0.,  9., 16., 21., 24., 25., 24., 21., 16.,  9.,  0.], dtype=float32)